<a href="https://colab.research.google.com/github/estebandurangov/SO-Final-Project/blob/main/Scheduling_SO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scheduling algorithm prediction
# Students:
# - Miguel Angel Alvarez
# - Esteban Durango
# - Saray Cubillos García

# 1) Load libraries and data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from scipy.special import entr # Entropía de Shannon
from scipy.stats import iqr # Interquartile range
from sklearn.ensemble import IsolationForest # algoritmo de aislamiento forestal
from sklearn.model_selection import GridSearchCV

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,  log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_validate
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from collections import Counter

In [ ]:
data=pd.read_csv("/content/output.csv")
df_copy = data.copy()
#Revolver datos
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
data

,N_Jobs,J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,J11,J12,J13,J14,J15,Best_Policy,Quantum
0,15,1,46,12.0,33.0,41,49.0,60.0,21.0,31.0,5.0,57.0,50.0,36.0,52.0,45.0,SJF,0
1,9,4,41,23.0,3.0,58,21.0,50.0,4.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,SJF,0
2,4,3,3,6.0,9.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RR,8
3,8,1,23,7.0,18.0,49,11.0,35.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SJF,0
4,9,57,34,26.0,26.0,4,10.0,59.0,36.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,SJF,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,5,13,40,21.0,44.0,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MLQ,0
993,10,21,57,30.0,46.0,59,8.0,50.0,51.0,39.0,50.0,0.0,0.0,0.0,0.0,0.0,SJF,0
994,4,1,6,7.0,9.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RR,8
995,10,8,16,29.0,13.0,45,15.0,27.0,31.0,15.0,38.0,0.0,0.0,0.0,0.0,0.0,SJF,0


In [ ]:
#Separación de columnas
y = df_copy["Best_Policy"]
X = df_copy.drop("Best_Policy", axis=1)

#Codificar para que queden de Strings a numeros
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

#Division de datos
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.30, random_state=42, stratify=None)

#Estandarización
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## XGBoost

In [ ]:
model = XGBClassifier(random_state=42, eval_metric='logloss')
#Entrenamiento
model.fit(X_train, y_train)           #Entrenamiento del modelo
y_pred_final = model.predict(X_test)
accuracy = model.score(X_test, y_test)

# Decodificar las predicciones para obtener los strings originales
y_pred_strings = label_encoder.inverse_transform(y_pred_final)


# Contar la frecuencia de cada predicción
conteo = Counter(y_pred_strings)

# Encontrar la clase más frecuente y su frecuencia
mayor = 0
valor_mayor = None
for valor, veces in conteo.items():
    if mayor < veces:
        mayor = veces
        valor_mayor = valor

print(f"{valor_mayor}: {mayor} veces")
print("Exactitud del modelo:", round(accuracy, 4))
y_pred_strings

SJF: 166 veces
Exactitud del modelo: 0.8833


array(['SJF', 'SJF', 'RR', 'SJF', 'SJF', 'MLQ', 'MLQ', 'RR', 'RR', 'SJF',
       'MLQ', 'RR', 'MLQ', 'SJF', 'SJF', 'MLQ', 'SJF', 'SJF', 'MLQ',
       'MLQ', 'MLQ', 'SJF', 'MLQ', 'MLQ', 'MLQ', 'MLQ', 'SJF', 'SJF',
       'SJF', 'SJF', 'MLQ', 'SJF', 'SJF', 'SJF', 'MLQ', 'RR', 'SJF',
       'SJF', 'RR', 'RR', 'SJF', 'MLQ', 'SJF', 'SJF', 'MLQ', 'MLQ', 'SJF',
       'RR', 'RR', 'SJF', 'RR', 'SJF', 'MLQ', 'MLQ', 'SJF', 'RR', 'MLQ',
       'SJF', 'MLQ', 'MLQ', 'SJF', 'SJF', 'SJF', 'MLQ', 'SJF', 'RR', 'RR',
       'RR', 'SJF', 'SJF', 'MLQ', 'MLQ', 'MLQ', 'SJF', 'SJF', 'MLQ',
       'MLQ', 'RR', 'SJF', 'RR', 'SJF', 'MLQ', 'SJF', 'SJF', 'RR', 'SJF',
       'MLQ', 'SJF', 'SJF', 'SJF', 'SJF', 'MLQ', 'SJF', 'SJF', 'SJF',
       'RR', 'SJF', 'RR', 'RR', 'MLQ', 'SJF', 'SJF', 'SJF', 'MLQ', 'SJF',
       'SJF', 'SJF', 'MLQ', 'SJF', 'RR', 'SJF', 'SJF', 'MLQ', 'SJF',
       'SJF', 'MLQ', 'SJF', 'SJF', 'SJF', 'MLQ', 'RR', 'SJF', 'SJF',
       'MLQ', 'SJF', 'SJF', 'RR', 'SJF', 'SJF', 'SJF', 'SJF', 'MLQ',
 

## Random Forest

In [ ]:
random_forest = RandomForestClassifier(random_state=42)
#Entrenamiento
random_forest.fit(X_train, y_train)
y_pred_final = model.predict(X_test)

# Decodificar las predicciones para obtener los strings originales
y_pred_strings = label_encoder.inverse_transform(y_pred_final)

accuracy = random_forest.score(X_test, y_test)
print("Exactitud del modelo:", round(accuracy, 4))


conteo = Counter(y_pred_strings)

# Encontrar la clase más frecuente y su frecuencia
mayor = 0
valor_mayor = None
for valor, veces in conteo.items():
    if mayor < veces:
        mayor = veces
        valor_mayor = valor

print(f"{valor_mayor}: {mayor} veces")

SJF: 166 veces
Exactitud del modelo: 0.9033


## SVM

In [ ]:
svm_classifier = SVC(kernel='linear', random_state=23)

#Entrenamiento
svm_classifier.fit(X_train, y_train)
y_pred_final = model.predict(X_test)

# Decodificar las predicciones para obtener los strings originales
y_pred_strings = label_encoder.inverse_transform(y_pred_final)

conteo = Counter(y_pred_strings)

# Encontrar la clase más frecuente y su frecuencia
mayor = 0
valor_mayor = None
for valor, veces in conteo.items():
    if mayor < veces:
        mayor = veces
        valor_mayor = valor

print(f"{valor_mayor}: {mayor} veces")

accuracy = svm_classifier.score(X_test, y_test)
print("Exactitud del modelo SVM:", round(accuracy, 4))


SJF: 166 veces
Exactitud del modelo SVM: 0.9167


In [ ]:
# Imprimir los resultados
for key, value in conteo.items():
    print(f"{key}: {value} veces")

SJF: 166 veces
RR: 55 veces
MLQ: 79 veces
